In [1]:
import warnings
warnings.resetwarnings()

import scprep
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import random

import os

from tqdm import tqdm

from deepimpute.multinet import MultiNet

from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import roc_auc_score

2023-08-21 15:55:51.027604: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-21 15:55:55.346475: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def get_data_for_i(i):
    original_ = pd.read_csv('../../data/mid_simulation/data.csv.gz', index_col=0)
    df_ = pd.read_csv('../../data/mid_simulation/drp_{}0.csv.gz'.format(i), index_col=0)
    df_.index = [int(i) for i in df_.index]
    df_.columns = [int(i) for i in df_.columns]

    original_.columns = df_.columns
    original_.index = df_.index

    n = original_.size
    original_val = original_.values.copy()
    t = list(np.ndindex(original_.shape))
    random.Random(42).shuffle(t)

    mask = t[:int(len(t)/10 * i)]

    thr = np.sum(np.sign(df_)) > 0
    original_ = original_.loc[:, list(thr)]
    df_ = df_.loc[:, list(thr)]

    # original = original_.values
    original = np.log(original_+1)

    # df = df_.values
    df = np.log(df_+1)

    tmp = pd.DataFrame(thr)
    remove = [int(i) for i in tmp[tmp[0] == False].index]
    mask = [i for i in mask if i[1] not in remove]
    
    return df_, mask, original

In [3]:
mses = {}
corrs = {}
mses_ = {}
corrs_ = {}
mses__ = {}
corrs__ = {}
aucs = {}
method = 'DeepImpute'

for i in (range(9)):
    print(i)
    df_, mask, original = get_data_for_i(i+1)

    multinet = MultiNet() 
    multinet.fit(df_)
    
    pred = multinet.predict(df_)
    pred = np.log(pred.values+1)
    
    pred = pd.DataFrame(pred, index=df_.index, columns=df_.columns)
#     pred.to_csv('/export/scratch/inoue019/cell_result/{}_{}.csv.gz'.format(method, i), compression='gzip')

    origin = np.array([original.loc[i] for i in mask])
    predict = np.array([pred.loc[i] for i in mask])

    mses[i] = mse(origin, predict)
    corrs[i] = np.corrcoef(origin, predict)[0][1]
    mses_[i] = mse(origin[origin != 0], predict[origin != 0])
    corrs_[i] = np.corrcoef(origin[origin != 0], predict[origin != 0])[0][1]
    mses__[i] = mse(origin[origin == 0], predict[origin == 0])
    
    df =  pd.DataFrame(np.array(predict))
    df['rank'] = df.rank()
    df['label'] = np.sign(origin)
    aucs[i] = roc_auc_score(df['label'], df['rank'])

0
Using all the cores (20)
Input dataset is 5000 cells (rows) and 5000 genes (columns)
First 3 rows and columns:
   0  1  2
0  0  7  2
1  1  4  0
2  0  0  0
5120 genes selected for imputation
Net 0: 730 predictors, 512 targets
Net 1: 727 predictors, 512 targets
Net 2: 768 predictors, 512 targets
Net 3: 720 predictors, 512 targets
Net 4: 760 predictors, 512 targets
Net 5: 740 predictors, 512 targets
Net 6: 735 predictors, 512 targets
Net 7: 728 predictors, 512 targets
Net 8: 776 predictors, 512 targets
Net 9: 690 predictors, 512 targets
Normalization
Building network
[{'type': 'dense', 'neurons': 256, 'activation': 'relu'}, {'type': 'dropout', 'rate': 0.2}]


2023-08-21 15:56:37.889737: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/inoue019/.local/lib/python3.10/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Fitting with 5000 cells
Epoch 1/500
75/75 [==============================] - 3s 14ms/step - loss: 4.1688 - dense_10_loss: 0.4202 - dense_11_loss: 0.4271 - dense_12_loss: 0.4221 - dense_13_loss: 0.4713 - dense_14_loss: 0.3736 - dense_15_loss: 0.3928 - dense_16_loss: 0.4179 - dense_17_loss: 0.4488 - dense_18_loss: 0.3609 - dense_19_loss: 0.4341 - val_loss: 2.0641 - val_dense_10_loss: 0.1951 - val_dense_11_loss: 0.2162 - val_dense_12_loss: 0.2128 - val_dense_13_loss: 0.2236 - val_dense_14_loss: 0.1974 - val_dense_15_loss: 0.2023 - val_dense_16_loss: 0.2069 - val_dense_17_loss: 0.2159 - val_dense_18_loss: 0.1826 - val_dense_19_loss: 0.2112
Epoch 2/500
75/75 [==============================] - 1s 9ms/step - loss: 2.2840 - dense_10_loss: 0.2174 - dense_11_loss: 0.2408 - dense_12_loss: 0.2299 - dense_13_loss: 0.2547 - dense_14_loss: 0.2126 - dense_15_loss: 0.2187 - dense_16_loss: 0.2297 - dense_17_loss: 0.2379 - dense_18_loss: 0.2048 - dense_19_loss: 0.2376 - val_loss: 1.6614 - val_dense_10_lo

75/75 [==============================] - 1s 8ms/step - loss: 1.3197 - dense_10_loss: 0.1263 - dense_11_loss: 0.1366 - dense_12_loss: 0.1359 - dense_13_loss: 0.1436 - dense_14_loss: 0.1246 - dense_15_loss: 0.1273 - dense_16_loss: 0.1346 - dense_17_loss: 0.1352 - dense_18_loss: 0.1221 - dense_19_loss: 0.1334 - val_loss: 1.1983 - val_dense_10_loss: 0.1126 - val_dense_11_loss: 0.1252 - val_dense_12_loss: 0.1270 - val_dense_13_loss: 0.1276 - val_dense_14_loss: 0.1141 - val_dense_15_loss: 0.1157 - val_dense_16_loss: 0.1198 - val_dense_17_loss: 0.1240 - val_dense_18_loss: 0.1120 - val_dense_19_loss: 0.1202
Epoch 15/500
75/75 [==============================] - 1s 8ms/step - loss: 1.2995 - dense_10_loss: 0.1243 - dense_11_loss: 0.1344 - dense_12_loss: 0.1341 - dense_13_loss: 0.1409 - dense_14_loss: 0.1235 - dense_15_loss: 0.1254 - dense_16_loss: 0.1324 - dense_17_loss: 0.1325 - dense_18_loss: 0.1204 - dense_19_loss: 0.1315 - val_loss: 1.1914 - val_dense_10_loss: 0.1120 - val_dense_11_loss: 0.12

75/75 [==============================] - 1s 8ms/step - loss: 1.1497 - dense_10_loss: 0.1099 - dense_11_loss: 0.1190 - dense_12_loss: 0.1186 - dense_13_loss: 0.1245 - dense_14_loss: 0.1095 - dense_15_loss: 0.1107 - dense_16_loss: 0.1172 - dense_17_loss: 0.1175 - dense_18_loss: 0.1068 - dense_19_loss: 0.1160 - val_loss: 1.1219 - val_dense_10_loss: 0.1050 - val_dense_11_loss: 0.1168 - val_dense_12_loss: 0.1192 - val_dense_13_loss: 0.1188 - val_dense_14_loss: 0.1072 - val_dense_15_loss: 0.1089 - val_dense_16_loss: 0.1128 - val_dense_17_loss: 0.1154 - val_dense_18_loss: 0.1059 - val_dense_19_loss: 0.1120
Epoch 28/500
75/75 [==============================] - 1s 8ms/step - loss: 1.1412 - dense_10_loss: 0.1089 - dense_11_loss: 0.1178 - dense_12_loss: 0.1178 - dense_13_loss: 0.1237 - dense_14_loss: 0.1089 - dense_15_loss: 0.1101 - dense_16_loss: 0.1163 - dense_17_loss: 0.1170 - dense_18_loss: 0.1056 - dense_19_loss: 0.1151 - val_loss: 1.1197 - val_dense_10_loss: 0.1053 - val_dense_11_loss: 0.11

...

75/75 [==============================] - 2s 29ms/step - loss: 0.2095 - dense_129_loss: 0.0440 - dense_130_loss: 0.0425 - dense_131_loss: 0.0386 - dense_132_loss: 0.0431 - dense_133_loss: 0.0413 - val_loss: 0.2806 - val_dense_129_loss: 0.0575 - val_dense_130_loss: 0.0568 - val_dense_131_loss: 0.0510 - val_dense_132_loss: 0.0585 - val_dense_133_loss: 0.0567
Epoch 24/500
75/75 [==============================] - 2s 28ms/step - loss: 0.2048 - dense_129_loss: 0.0430 - dense_130_loss: 0.0416 - dense_131_loss: 0.0378 - dense_132_loss: 0.0421 - dense_133_loss: 0.0403 - val_loss: 0.2796 - val_dense_129_loss: 0.0573 - val_dense_130_loss: 0.0566 - val_dense_131_loss: 0.0508 - val_dense_132_loss: 0.0583 - val_dense_133_loss: 0.0566
Epoch 25/500
75/75 [==============================] - 1s 7ms/step - loss: 0.2005 - dense_129_loss: 0.0421 - dense_130_loss: 0.0408 - dense_131_loss: 0.0370 - dense_132_loss: 0.0412 - dense_133_loss: 0.0394 - val_loss: 0.2787 - val_dense_129_loss: 0.0570 - val_dense_130_l

75/75 [==============================] - 1s 10ms/step - loss: 0.1438 - dense_129_loss: 0.0302 - dense_130_loss: 0.0292 - dense_131_loss: 0.0264 - dense_132_loss: 0.0296 - dense_133_loss: 0.0283 - val_loss: 0.2723 - val_dense_129_loss: 0.0551 - val_dense_130_loss: 0.0554 - val_dense_131_loss: 0.0496 - val_dense_132_loss: 0.0565 - val_dense_133_loss: 0.0558
Epoch 46/500
75/75 [==============================] - 2s 26ms/step - loss: 0.1422 - dense_129_loss: 0.0300 - dense_130_loss: 0.0288 - dense_131_loss: 0.0262 - dense_132_loss: 0.0293 - dense_133_loss: 0.0280 - val_loss: 0.2721 - val_dense_129_loss: 0.0550 - val_dense_130_loss: 0.0554 - val_dense_131_loss: 0.0495 - val_dense_132_loss: 0.0563 - val_dense_133_loss: 0.0558
Epoch 47/500
75/75 [==============================] - 2s 26ms/step - loss: 0.1405 - dense_129_loss: 0.0296 - dense_130_loss: 0.0284 - dense_131_loss: 0.0259 - dense_132_loss: 0.0291 - dense_133_loss: 0.0276 - val_loss: 0.2722 - val_dense_129_loss: 0.0550 - val_dense_130_

/home/inoue019/.local/lib/python3.10/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Fitting with 5000 cells
Epoch 1/500
75/75 [==============================] - 4s 37ms/step - loss: 0.7315 - dense_138_loss: 0.1945 - dense_139_loss: 0.1675 - dense_140_loss: 0.1727 - dense_141_loss: 0.1968 - val_loss: 0.6710 - val_dense_138_loss: 0.1790 - val_dense_139_loss: 0.1465 - val_dense_140_loss: 0.1578 - val_dense_141_loss: 0.1877
Epoch 2/500
75/75 [==============================] - 1s 18ms/step - loss: 0.4955 - dense_138_loss: 0.1315 - dense_139_loss: 0.1114 - dense_140_loss: 0.1161 - dense_141_loss: 0.1364 - val_loss: 0.3159 - val_dense_138_loss: 0.0824 - val_dense_139_loss: 0.0673 - val_dense_140_loss: 0.0749 - val_dense_141_loss: 0.0914
Epoch 3/500
75/75 [==============================] - 1s 19ms/step - loss: 0.2320 - dense_138_loss: 0.0600 - dense_139_loss: 0.0519 - dense_140_loss: 0.0557 - dense_141_loss: 0.0643 - val_loss: 0.1889 - val_dense_138_loss: 0.0464 - val_dense_139_loss: 0.0416 - val_dense_140_loss: 0.0470 - val_dense_141_loss: 0.0539
Epoch 4/500
75/75 [=========

Epoch 27/500
75/75 [==============================] - 1s 17ms/step - loss: 0.0867 - dense_138_loss: 0.0225 - dense_139_loss: 0.0199 - dense_140_loss: 0.0214 - dense_141_loss: 0.0228 - val_loss: 0.1493 - val_dense_138_loss: 0.0378 - val_dense_139_loss: 0.0343 - val_dense_140_loss: 0.0373 - val_dense_141_loss: 0.0398
Epoch 28/500
75/75 [==============================] - 1s 16ms/step - loss: 0.0843 - dense_138_loss: 0.0220 - dense_139_loss: 0.0194 - dense_140_loss: 0.0208 - dense_141_loss: 0.0222 - val_loss: 0.1487 - val_dense_138_loss: 0.0377 - val_dense_139_loss: 0.0341 - val_dense_140_loss: 0.0372 - val_dense_141_loss: 0.0397
Epoch 29/500
75/75 [==============================] - 1s 17ms/step - loss: 0.0826 - dense_138_loss: 0.0214 - dense_139_loss: 0.0191 - dense_140_loss: 0.0203 - dense_141_loss: 0.0218 - val_loss: 0.1485 - val_dense_138_loss: 0.0377 - val_dense_139_loss: 0.0341 - val_dense_140_loss: 0.0372 - val_dense_141_loss: 0.0395
Epoch 30/500
75/75 [=============================

In [4]:
pd.DataFrame([
    mses.values(),
    mses_.values(),
    mses__.values(),
    corrs.values(),
    corrs_.values(),
    aucs.values()
], index=['mse', 'mse (nonzero)', 'mse (zero)', 'corr', 'corrs (nonzero)', 'auc'])

,0,1,2,3,4,5,6,7,8
mse,0.972124,0.969750,0.913628,0.833689,0.838255,0.764896,0.632347,0.576384,0.539399
mse (nonzero),0.261543,0.264833,0.295930,0.343280,0.350957,0.404118,0.532147,0.617113,0.732938
mse (zero),1.334862,1.329229,1.228330,1.083496,1.086327,0.948594,0.683383,0.555630,0.440781
corr,0.651784,0.645155,0.562575,0.524388,0.523665,0.514200,0.531789,0.540342,0.541148
corrs (nonzero),0.739414,0.732018,0.694569,0.655082,0.651728,0.625183,0.604793,0.596924,0.588316
auc,0.700447,0.698002,0.703903,0.709858,0.707285,0.711091,0.712592,0.705064,0.687848
